In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
additional_info = pd.read_json('/content/gdrive/MyDrive/Графы/additional_info.json')
game_info = pd.read_json('/content/gdrive/MyDrive/Графы/games_info.json')

In [ ]:
additional_info

,genres,tags
4,"[Simulation, Combat, Sci-fi]","[Science, Classic, Good Old Game, Space, Sci-f..."
5,"[Simulation, Combat, Sci-fi]","[Science, Classic, Multiplayer, Good Old Game,..."
6,"[Action, TPP, Sci-fi]","[Science, Classic, Good Old Game, Third Person]"
7,"[Action, TPP, Sci-fi]","[Science, Classic, Atmospheric, Dark, Good Old..."
8,"[Role-playing, Adventure, Fantasy]","[Fantasy, Classic, Story Rich, Atmospheric, Ho..."
...,...,...
2147483119,"[Adventure, Point-and-click, Modern]","[Point&Click, Modern]"
2147483128,"[Simulation, Building, Managerial]",[]
2147483134,"[Role-playing, Action, Fantasy]",[Fantasy]
2147483137,"[Action, Combat, Open World]","[Fantasy, Science, Exploration, Open World, Th..."


In [ ]:
game_info_cut = game_info.drop(['is_secret',
                                'is_installable', 
                                'purchase_link', 
                                'links', 
                                'images', 
                                'downloads', 
                                'screenshots', 
                                'videos', 
                                'expanded_dlcs', 
                                'related_products'
                                ], 
                               axis=1)

Разделяем json значения на несколько столбцов



In [ ]:
def normalize_column(df, column):
  normalized_column = pd.json_normalize(df[column]).set_index(df.index)
  df_dropped = df.drop([column], axis = 1)
  return pd.concat([df_dropped, normalized_column], axis = 1)

In [ ]:
game_info_cut = normalize_column(game_info_cut, 'content_system_compatibility')
game_info_cut = normalize_column(game_info_cut, 'description')
game_info_cut = normalize_column(game_info_cut, 'in_development')

Удаляем html теги

In [ ]:
from bs4 import BeautifulSoup

def remove_tags_from_text(text):
    return BeautifulSoup(text, 'lxml').text

In [ ]:
def remove_tags_from_column(df, column):
  df[column] = df[column].map(remove_tags_from_text, na_action='ignore')
  return df

In [ ]:
game_info_cut = remove_tags_from_column(game_info_cut, 'lead')
game_info_cut = remove_tags_from_column(game_info_cut, 'full')
game_info_cut = remove_tags_from_column(game_info_cut, 'whats_cool_about_it')
game_info_cut = remove_tags_from_column(game_info_cut, 'changelog')

/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


Берем из dlcs только id. Проверяем их, чтобы они были во фрейме

In [ ]:
import numpy as np

def id_map(products):
  ids = []
  for product in products:
    id = product['id']
    if id in game_info_cut.index:
      ids.append(id)

  if len(ids) == 0:
    return np.nan
  else:
    return ids
    

def extract_ids(df):
  filtred_df = df[df.dlcs.map(len) > 1]
  normalized_dlcs = pd.json_normalize(filtred_df.dlcs).set_index(filtred_df.index)
  df['dlcs'] = normalized_dlcs.products.map(id_map)
  return df

In [ ]:
game_info_cut = extract_ids(game_info_cut)

Заменяем пустые строки на NaN

In [ ]:
game_info_cut = game_info_cut.replace(r'^\s*$', np.nan, regex=True)

Добавляем теги и жанр

In [ ]:
game_info_cut = pd.concat([game_info_cut, additional_info], axis = 1)

Удаляем строки, которые не пропарсились с сайта

In [ ]:
game_info_cut = game_info_cut[game_info_cut['title'].notna()] 

In [ ]:
!pip install python-slugify

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from slugify import slugify

def slugify_map(items):    
  if len(items) == 0:
    return np.nan

  slugify_items = []
  for item in items:
    slugify_items.append(slugify(item))

  return slugify_items

In [ ]:
game_info_cut.genres = game_info_cut[game_info_cut.genres.notna()].genres.map(slugify_map)
game_info_cut.tags = game_info_cut[game_info_cut.tags.notna()].tags.map(slugify_map)

In [ ]:
game_info_cut

,title,slug,languages,game_type,is_pre_order,release_date,dlcs,changelog,windows,osx,linux,lead,full,whats_cool_about_it,active,until,genres,tags
4,Descent: Freespace Battle Pack,freespace_expansion,{'en': 'English'},game,False,2008-07-15T00:00:00+0300,NaN,Patch gog-6 (03 November 2015)\n\nWe've upload...,True,True,False,The heartless Shivans emerged through subspace...,The heartless Shivans emerged through subspace...,Includes the original Freespace and the Silent...,False,None,"[simulation, combat, sci-fi]","[science, classic, good-old-game, space, sci-f..."
5,Freespace 2,freespace_2,"{'en': 'English', 'pl': 'polski'}",game,False,2008-07-15T00:00:00+0300,NaN,Internal Update (08 November 2019)\n\nadded Po...,True,False,False,"The year is 2367, thirty two years after the G...","The year is 2367, thirty two years after the G...",Without a doubt one of the best space sims eve...,False,None,"[simulation, combat, sci-fi]","[science, classic, multiplayer, good-old-game,..."
6,MDK,mdk,"{'de': 'Deutsch', 'en': 'English', 'es': 'espa...",game,False,1991-12-25T00:00:00+0200,NaN,Update (08 November 2019)\n\n\nadded text supp...,True,True,False,As vast alien mining cities suck the power and...,As vast alien mining cities suck the power and...,One of the greatest and most memorable action ...,False,None,"[action, tpp, sci-fi]","[science, classic, good-old-game, third-person]"
7,Messiah,messiah,{'en': 'English'},game,False,2008-07-15T00:00:00+0300,NaN,NaN,True,False,False,Bob is a working-class angel ordered by God hi...,Bob is a working-class angel ordered by God hi...,Possess anyone you want and make them do your ...,False,None,"[action, tpp, sci-fi]","[science, classic, atmospheric, dark, good-old..."
8,Lionheart: Legacy of the Crusader,lionheart_legacy_of_the_crusader,"{'de': 'Deutsch', 'en': 'English', 'es': 'espa...",game,False,2008-07-15T00:00:00+0300,NaN,Update (26 November 2021)\nAdded languages: Fr...,True,False,False,Award-winning Black Isle Studios unleashes an ...,Award-winning Black Isle Studios unleashes an ...,An original story set in a warped alternative ...,False,None,"[role-playing, adventure, fantasy]","[fantasy, classic, story-rich, atmospheric, ho..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2147483119,Leisure Suit Larry - Wet Dreams Dry Twice,leisure_suit_larry_wet_dreams_dry_twice,"{'de': 'Deutsch', 'en': 'English', 'es': 'espa...",game,False,2020-10-23T12:50:00+0300,NaN,Update (26 November 2020)\nThe Update includes...,True,True,False,"Hey Ladies, it’s time to get wet again! I am n...","Hey Ladies, it’s time to get wet again! I am n...",NaN,False,None,"[adventure, point-and-click, modern]","[point-click, modern]"
2147483128,Parkitect - Booms & Blooms,parkitect_booms_blooms,"{'cn': '中文(简体)', 'de': 'Deutsch', 'en': 'Engli...",dlc,False,2020-09-02T15:50:00+0300,NaN,NaN,False,False,False,Welcome to Parkitect: Booms & Blooms!\n\n\nEff...,Welcome to Parkitect: Booms & Blooms!\n\n\nEff...,NaN,False,None,"[simulation, building, managerial]",NaN
2147483134,Warhammer: Chaosbane - Witch Hunter DLC,warhammer_chaosbane_witch_hunter,"{'br': 'Português do Brasil', 'cn': '中文(简体)', ...",dlc,False,2020-11-10T18:50:00+0200,NaN,NaN,False,False,False,By purchasing this DLC you unlock Jurgen Haide...,By purchasing this DLC you unlock Jurgen Haide...,NaN,False,None,"[role-playing, action, fantasy]",[fantasy]
2147483137,The Falconeer,the_falconeer,"{'br': 'Português do Brasil', 'cn': '中文(简体)', ...",game,False,2020-11-10T15:50:00+0200,"[1985033417, 1184972632, 1645580324, 1423752095]",The Falconeer Update 1.4.3.0\n\n\nAdded Portug...,True,False,False,"\n\nFROM THE CLOUDS THEY DIVE, THE SCREAMS OF ...","\n\nFROM THE CLOUDS THEY DIVE, THE SCREAMS OF ...",NaN,False,None,"[action, combat, open-world]","[fantasy, science, exploration, open-world, th..."


In [ ]:
!pip install python-dateutil

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from dateutil import parser

def remove_timezone(datetime):
  datetime = parser.parse(datetime).replace(tzinfo=None)
  return datetime.isoformat()

In [ ]:
game_info_cut.release_date = game_info_cut[game_info_cut.release_date.notnull()].release_date.map(remove_timezone)
game_info_cut.until = game_info_cut[game_info_cut.until.notnull()].until.map(remove_timezone)

In [ ]:
game_info_cut.to_json('/content/gdrive/MyDrive/Графы/games_info_finished.json')